# Multi-agent Strategy B, Mixed Strategy.

In [2]:
import pandas as pd
import numpy as np
import time
import math

In [3]:
# Imports

validation = pd.read_csv('we_data/validation.csv')
train = pd.read_csv('we_data/train.csv')
test = pd.read_csv('we_data/test.csv')

test_pctr = pd.read_csv('pctr_test_8707.csv')
val_pctr = pd.read_csv('pctr_validation_8707.csv')

In [4]:
data = validation
pred = val_pctr
pred = pred['pctr']

In [5]:
# Constants

limited_budget = 6250*1000
min_bid = min(data['bidprice'])
max_bid = max(data['bidprice'])

print(min_bid, max_bid)

min_pay = min(data['payprice'])
max_pay = max(data['payprice'])

print(min_pay, max_pay)

avgCTR = (data.click.sum() / len(validation))

print(avgCTR)

c = 6
ld = 1e-6

agent_num = 15

227 300
0 300
0.0006646376573167722


In [6]:
def binary_search(arr, val):
    # returns index
    low, high = 0, len(arr) - 1
    
    while low < high:
        
        mid = int((low+high)/2)
        
        if arr[mid] < val:
            low = mid + 1
        else:
            high = mid
    
    return low

In [8]:
#merge prediction

data_new = data.merge(val_pctr)

# rename column label
columns = list(data_new.columns)
columns[-1] = 'pCTR'
data_new.columns = columns

print(data_new.columns)
data_new.head()

Index(['click', 'weekday', 'hour', 'bidid', 'userid', 'useragent', 'IP',
       'region', 'city', 'adexchange', 'domain', 'url', 'urlid', 'slotid',
       'slotwidth', 'slotheight', 'slotvisibility', 'slotformat', 'slotprice',
       'creative', 'bidprice', 'payprice', 'keypage', 'advertiser', 'usertag',
       'pCTR'],
      dtype='object')


,click,weekday,hour,bidid,userid,useragent,IP,region,city,adexchange,...,slotvisibility,slotformat,slotprice,creative,bidprice,payprice,keypage,advertiser,usertag,pCTR
0,0,4,20,bbcb813b6166538503d8b33a5602d7d72f6019dc,663169f66491c98c69f1f94a5c48fa34aa9fe06f,windows_ie,211.144.203.*,79,79,1.0,...,2,1,0,cb7c76e7784031272e37af8e7e9b062c,300,23,bebefa5efe83beee17a3d245e7c5085b,1458,"13866,10111",0.000178
1,0,1,21,5a07316c49477cb5d9b4d5aa39c27d6c3be7f92d,a23f0cfab6592c137f796e68fa752fceba08bb1b,windows_chrome,58.247.250.*,79,79,1.0,...,0,1,0,c938195f9e404b4f38c7e71bf50263e5,238,75,a3becb967c015e9acb5e72f22b100ef8,3476,"10075,10057,10024,10052,10083,10063,10006,10110",0.000123
2,0,4,8,f6ece71dae81d6b16bfb24ad6dd5611472d4c673,5105a013ea54a9706146033e6a138d5234a3803e,windows_ie,117.12.111.*,2,2,2.0,...,2,0,5,911b2d84826786018761e8c0b0a3a60c,238,65,d29e59bf0f7f8243858b8183f14d4412,3358,"13866,10024,10059,10063,13776,10083,10006,10111",0.000153
3,0,5,15,b4d5c57c9b38ff5a12954fa01e11931b4e6bfbbb,577e9a768ce17e73c61ba7022db927f9ee761830,windows_chrome,61.187.224.*,201,205,2.0,...,2,0,5,dc0998c10f8f0b623b5d949e8272e4c7,238,6,d29e59bf0f7f8243858b8183f14d4412,3358,"13866,10057,10006,10063,10110",0.000110
4,0,1,18,0899bf144249458ea9c89188473694bf44c7ca15,e4bf79c562745d671b19f2edf7fda89c2e25987f,windows_chrome,117.41.145.*,134,135,2.0,...,1,0,5,23485fcd23122d755d38f8c89d46ca56,249,5,7e0e909ea89f707bc20f9585c9072081,3476,"10006,10063,13776,10110",0.000140


# Creating bidding functions for multiple agents

In [102]:
final_agent_arr = []

In [103]:
def constant_bidding_bids(num_agent):
    base_value = 80
    difference_val = 5
    base_bid_const = []
    for i in range(num_agent):
        base_bid_const.append(base_value + i*difference_val)
    return base_bid_const
        
def constant_bidding(dat, bid):
    budget = np.cumsum(dat.payprice*(dat.payprice <= bid))
    fin = binary_search(budget, limited_budget)
#     print(budget[fin], budget[fin], budget[fin])

    n_clk = np.sum((dat.click*(dat.payprice <= bid))[:fin]) 
    n_im = np.sum((dat.payprice <= bid)[:fin])
    cost = budget[fin]
    
    agent_type = 'constant'
        
    # labels = ['agent_type', 'base_bid', 'bid', 'clicks', 'imps_won', 'total_spend']
    result = [agent_type, bid, bid, n_clk, n_im, cost]
    return result

In [104]:
# Create Constant Agents:
agents_base = constant_bidding_bids(agent_num)
const_agent_arr = []
for i in range(agent_num):
    temp = constant_bidding(data, agents_base[i])
    print(temp)
    const_agent_arr.append(temp)
    
print(const_agent_arr)

['constant', 80, 80, 66, 140549, 6250022]
['constant', 85, 85, 66, 136112, 6250001]
['constant', 90, 90, 65, 130537, 6250025]
['constant', 95, 95, 65, 127952, 6250018]
['constant', 100, 100, 62, 125767, 6250021]
['constant', 105, 105, 59, 123195, 6250045]
['constant', 110, 110, 57, 121348, 6250043]
['constant', 115, 115, 56, 119406, 6250003]
['constant', 120, 120, 57, 117427, 6250048]
['constant', 125, 125, 58, 115680, 6250048]
['constant', 130, 130, 57, 113729, 6250006]
['constant', 135, 135, 57, 112121, 6250054]
['constant', 140, 140, 59, 110432, 6250090]
['constant', 145, 145, 58, 108876, 6250014]
['constant', 150, 150, 55, 106230, 6250023]
[['constant', 80, 80, 66, 140549, 6250022], ['constant', 85, 85, 66, 136112, 6250001], ['constant', 90, 90, 65, 130537, 6250025], ['constant', 95, 95, 65, 127952, 6250018], ['constant', 100, 100, 62, 125767, 6250021], ['constant', 105, 105, 59, 123195, 6250045], ['constant', 110, 110, 57, 121348, 6250043], ['constant', 115, 115, 56, 119406, 62500

In [105]:
# Create final array
final_agent_arr_const = []

for item in const_agent_arr:    
    bid_values = np.full(len(pred), item[2])
    final_agent_arr_const.append(bid_values)
    
final_agent_arr_const = np.array(final_agent_arr_const)
print(final_agent_arr_const)

[[ 80  80  80 ...  80  80  80]
 [ 85  85  85 ...  85  85  85]
 [ 90  90  90 ...  90  90  90]
 ...
 [140 140 140 ... 140 140 140]
 [145 145 145 ... 145 145 145]
 [150 150 150 ... 150 150 150]]


In [106]:
upper = 110
lower = 60

def random_bidding(dat):     
    n = data.shape[0]
    bid = np.random.randint(lower, upper)
    
    budget = np.cumsum(dat['payprice']*(dat['payprice'] < bid))

    end = binary_search(budget, limited_budget)

    n_clk = np.sum((dat['click']*(dat['payprice'] < bid))[:end]) 
    n_im = np.sum((dat['payprice'] < bid)[:end])
    cost = budget[end-1]
    
    agent_type = 'random'
        
    # labels = ['agent_type', 'base_bid', 'bid', 'clicks', 'imps_won', 'total_spend']
    result = [agent_type, bid, bid, n_clk, n_im, cost]
    return result

In [12]:
# Create Random Agents:
rand_agent_arr = []
for i in range(agent_num):
    temp = random_bidding(data)
    print(temp)
    rand_agent_arr.append(temp)

['random', 136, 136, 57, 112121, 6249979]
['random', 282, 282, 51, 80638, 6249985]
['random', 299, 299, 51, 79832, 6249929]
['random', 298, 298, 51, 79851, 6249922]
['random', 73, 73, 63, 160592, 6249979]
['random', 136, 136, 57, 112121, 6249979]
['random', 212, 212, 57, 89115, 6249990]
['random', 127, 127, 57, 115245, 6249923]
['random', 248, 248, 53, 83668, 6249976]
['random', 139, 139, 58, 111480, 6249985]
['random', 148, 148, 58, 108236, 6249972]
['random', 252, 252, 54, 83234, 6249979]
['random', 216, 216, 56, 88035, 6249969]
['random', 114, 114, 57, 120114, 6249953]
['random', 84, 84, 66, 138993, 6249974]


In [107]:
# Create final array
final_agent_arr_rand = []

for item in rand_agent_arr:    
    bid_values = np.full(len(pred), item[2])
    final_agent_arr_rand.append(bid_values)
    
final_agent_arr_rand = np.array(final_agent_arr_rand)
print(final_agent_arr_rand)

[[136 136 136 ... 136 136 136]
 [282 282 282 ... 282 282 282]
 [299 299 299 ... 299 299 299]
 ...
 [216 216 216 ... 216 216 216]
 [114 114 114 ... 114 114 114]
 [ 84  84  84 ...  84  84  84]]


In [108]:
def linear_bidding_base_bids(num_agent):
    base_value = 160
    difference_val = 5
    base_bid_linear = []
    for i in range(num_agent):
        base_bid_linear.append(base_value + i*difference_val)
    return base_bid_linear
        
def linear_bidding(dat, base_bid):
    bid = base_bid * (pred / avgCTR)

    budget = np.cumsum(dat.payprice*(dat.payprice <= bid))
    fin = binary_search(budget, limited_budget)
    #print(budget[fin], budget[fin], budget[fin])

    n_clk = np.sum((dat.click*(dat.payprice <= bid))[:fin]) 
    n_im = np.sum((dat.payprice <= bid)[:fin])
    cost = budget[fin]

    agent_type = 'linear'
    
    # labels = ['agent_type', 'base_bid', 'bid', 'clicks', 'imps_won', 'total_spend']
    result = [agent_type, base_bid, bid, n_clk, n_im, cost]

    return result

In [109]:
# Create Linear Agents:
agents_base = linear_bidding_base_bids(agent_num)
linear_agent_arr = []
for i in range(agent_num):
    temp = linear_bidding(data, agents_base[i])
    print(temp)
    linear_agent_arr.append(temp)

['linear', 160, 0          42.749790
1          29.580268
2          36.811877
3          26.491908
4          33.787794
5          58.387543
6          43.085852
7          67.356761
8          30.371075
9          38.423101
10         76.944421
11         51.099843
12         32.256252
13         53.002113
14         52.278952
15         32.237234
16         54.529080
17         58.077480
18         75.003635
19         71.158412
20         69.956193
21         73.445131
22         41.796970
23         32.994579
24         82.309871
25         44.707187
26         18.295683
27         77.831521
28         42.866304
29        157.637051
             ...    
303895    972.477188
303896     36.210768
303897     81.612709
303898    172.277690
303899     75.017356
303900     22.917750
303901     26.533315
303902     63.747456
303903     18.897515
303904     60.856016
303905     55.906793
303906     32.496744
303907     40.510735
303908     48.316010
303909     41.289746
303910     52.0601

['linear', 200, 0           53.437237
1           36.975335
2           46.014847
3           33.114886
4           42.234742
5           72.984429
6           53.857315
7           84.195952
8           37.963843
9           48.028877
10          96.180527
11          63.874804
12          40.320315
13          66.252641
14          65.348690
15          40.296543
16          68.161350
17          72.596850
18          93.754543
19          88.948015
20          87.445241
21          91.806414
22          52.246212
23          41.243224
24         102.887339
25          55.883984
26          22.869604
27          97.289402
28          53.582880
29         197.046313
             ...     
303895    1215.596485
303896      45.263460
303897     102.015887
303898     215.347112
303899      93.771695
303900      28.647188
303901      33.166643
303902      79.684320
303903      23.621894
303904      76.070020
303905      69.883491
303906      40.620930
303907      50.638419
303908      60.3

In [110]:
# Create final array
final_agent_arr_linear = []

for item in linear_agent_arr:    
    bid_values = np.array(item[2])
    final_agent_arr_linear.append(bid_values)
    
final_agent_arr_linear = np.array(final_agent_arr_linear)
print(final_agent_arr_linear)

[[ 42.74978958  29.58026796  36.81187747 ... 152.0118502   62.90729925
   56.34709317]
 [ 44.08572051  30.50465133  37.96224864 ... 156.76222052  64.87315235
   58.10793983]
 [ 45.42165143  31.42903471  39.11261981 ... 161.51259084  66.83900545
   59.86878649]
 ...
 [ 58.78096068  40.67286845  50.61633151 ... 209.01629402  86.49753647
   77.47725311]
 [ 60.1168916   41.59725182  51.76670269 ... 213.76666434  88.46338957
   79.23809977]
 [ 61.45282253  42.52163519  52.91707386 ... 218.51703466  90.42924267
   80.99894643]]


In [115]:
def sqr_bidding_base_bids(num_agent):
    base_value = 380
    difference_val = 5
    base_bid_sqr = []
    for i in range(num_agent):
        base_bid_sqr.append(base_value + i*difference_val)
    return base_bid_sqr

def sqr_bidding(dat, base_bid):
    bid = pd.Series(base_bid * ((pred / avgCTR)**2))

    budget = np.cumsum(dat.payprice*(dat.payprice <= bid))
    fin = binary_search(budget, limited_budget)
    #print(budget[fin], budget[fin], budget[fin])

    n_clk = np.sum((dat.click*(dat.payprice <= bid))[:fin]) 
    n_im = np.sum((dat.payprice <= bid)[:fin])
    cost = budget[fin]

    agent_type = 'sqr'
    
    # labels = ['agent_type', 'base_bid', 'bid', 'clicks', 'imps_won', 'total_spend']
    result = [agent_type, base_bid, bid, n_clk, n_im, cost]

    return result

In [116]:
# Create Square Agents:
agents_base = sqr_bidding_base_bids(agent_num)
sqr_agent_arr = []
for i in range(agent_num):
    temp = sqr_bidding(data, agents_base[i])
    print(temp)
    sqr_agent_arr.append(temp)

['sqr', 380, 0            27.127614
1            12.988166
2            20.114978
3            10.417659
4            16.945848
5            50.603906
6            27.555799
7            67.345103
8            13.691907
9            21.914343
10           87.881590
11           38.759910
12           15.444414
13           41.699418
14           40.569287
15           15.426208
16           44.136712
17           50.067874
18           83.504187
19           75.161618
20           72.643366
21           80.069968
22           25.931833
23           16.159534
24          100.565143
25           29.668687
26            4.968678
27           89.919663
28           27.275688
29          368.858871
              ...     
303895    14037.910735
303896       19.463418
303897       98.868791
303898      440.556597
303899       83.534743
303900        7.796283
303901       10.450249
303902       60.321114
303903        5.300942
303904       54.973156
303905       46.395172
303906       15.67557

['sqr', 425, 0            30.340094
1            14.526239
2            22.497015
3            11.651329
4            18.952593
5            56.596473
6            30.818986
7            75.320182
8            15.313318
9            24.509463
10           98.288621
11           43.349900
12           17.273358
13           46.637507
14           45.373544
15           17.252996
16           49.363427
17           55.996965
18           93.392840
19           84.062336
20           81.245870
21           89.551938
22           29.002708
23           18.073163
24          112.474173
25           33.182084
26            5.557075
27          100.568044
28           30.505704
29          412.539526
              ...     
303895    15700.294901
303896       21.768296
303897      110.576937
303898      492.727773
303899       93.427016
303900        8.719527
303901       11.687779
303902       67.464403
303903        5.928685
303904       61.483135
303905       51.889338
303906       17.53188

In [117]:
# Create final array
final_agent_arr_sqr = []

for item in sqr_agent_arr:    
    bid_values = np.array(item[2])
    final_agent_arr_sqr.append(bid_values)
    
final_agent_arr_sqr = np.array(final_agent_arr_sqr)
print(final_agent_arr_sqr)

[[ 27.12761381  12.98816625  20.11497822 ... 343.0034761   58.74159193
   47.12883067]
 [ 27.4845561   13.15906317  20.37964899 ... 347.51667974  59.51450762
   47.74894687]
 [ 27.84149839  13.3299601   20.64431976 ... 352.02988337  60.2874233
   48.36906306]
 ...
 [ 31.41092126  15.03892934  23.29102742 ... 397.1619197   68.01658013
   54.57022499]
 [ 31.76786354  15.20982627  23.55569818 ... 401.67512333  68.78949582
   55.19034118]
 [ 32.12480583  15.38072319  23.82036895 ... 406.18832696  69.5624115
   55.81045738]]


In [118]:
def exp_bidding_base_bids(num_agent):
    base_value = 20
    difference_val = 5
    base_bid_exp = []
    for i in range(num_agent):
        base_bid_exp.append(base_value + i*difference_val)
    return base_bid_exp

def exp_bidding(dat, base_bid):
    bid = base_bid * (np.exp(pred / avgCTR))

    budget = np.cumsum(dat.payprice*(dat.payprice <= bid))
    fin = binary_search(budget, limited_budget)
    #print(budget[fin], budget[fin], budget[fin])

    n_clk = np.sum((dat.click*(dat.payprice <= bid))[:fin]) 
    n_im = np.sum((dat.payprice <= bid)[:fin])
    cost = budget[fin]

    agent_type = 'exp'
    
    # labels = ['agent_type', 'base_bid', 'bid', 'clicks', 'imps_won', 'total_spend']
    result = [agent_type, base_bid, bid, n_clk, n_im, cost]

    return result
    

In [119]:
# Create Exponential Agents:
agents_base = exp_bidding_base_bids(agent_num)
exp_agent_arr = []
for i in range(agent_num):
    temp = exp_bidding(data, agents_base[i])
    print(temp)
    exp_agent_arr.append(temp)

['exp', 20, 0           26.125673
1           24.061401
2           25.173869
3           23.601416
4           24.702538
5           28.808037
6           26.180605
7           30.469069
8           24.180620
9           25.428654
10          32.350676
11          27.525320
12          24.467210
13          27.854526
14          27.728915
15          24.464302
16          28.121630
17          28.752264
18          31.960635
19          31.201693
20          30.968126
21          31.650829
22          25.970553
23          24.580376
24          33.453920
25          26.447250
26          22.422845
27          32.530538
28          26.144705
29          53.568641
             ...     
303895    8722.966886
303896      25.079470
303897      33.308470
303898      58.701650
303899      31.963376
303900      23.080041
303901      23.607524
303902      29.789437
303903      22.507346
303904      29.255933
303905      28.364822
303906      24.504014
303907      25.762614
303908      27.05055

['exp', 60, 0            78.377018
1            72.184204
2            75.521607
3            70.804247
4            74.107613
5            86.424112
6            78.541814
7            91.407206
8            72.541860
9            76.285963
10           97.052027
11           82.575959
12           73.401630
13           83.563579
14           83.186744
15           73.392906
16           84.364889
17           86.256793
18           95.881905
19           93.605078
20           92.904378
21           94.952487
22           77.911660
23           73.741128
24          100.361760
25           79.341751
26           67.268534
27           97.591614
28           78.434114
29          160.705923
              ...     
303895    26168.900659
303896       75.238409
303897       99.925409
303898      176.104950
303899       95.890128
303900       69.240123
303901       70.822573
303902       89.368312
303903       67.522038
303904       87.767798
303905       85.094467
303906       73.512041

In [120]:
# Create final array
final_agent_arr_exp = []

for item in exp_agent_arr:    
    bid_values = np.array(item[2])
    final_agent_arr_exp.append(bid_values)
    
final_agent_arr_exp = np.array(final_agent_arr_exp)
print(final_agent_arr_exp)

[[ 26.12567268  24.06140124  25.17386889 ...  51.71802347  29.63342342
   28.44298638]
 [ 32.65709085  30.07675155  31.46733611 ...  64.64752934  37.04177927
   35.55373298]
 [ 39.18850902  36.09210186  37.76080334 ...  77.57703521  44.45013513
   42.66447957]
 ...
 [104.50269072  96.24560497 100.69547556 ... 206.8720939  118.53369368
  113.77194553]
 [111.03410889 102.26095528 106.98894278 ... 219.80159977 125.94204953
  120.88269213]
 [117.56552706 108.27630559 113.28241001 ... 232.73110564 133.35040538
  127.99343872]]


In [121]:
def ortb_bidding(dat):
    bid = np.sqrt((c/ld)*np.asarray(pred)+(c*c-c))

    budget = np.cumsum(dat.payprice*(dat.payprice <= bid))
    fin = binary_search(budget, limited_budget)

    n_clk = np.sum((dat.click*(dat.payprice <= bid))[:fin]) 
    n_im = np.sum((dat.payprice <= bid)[:fin])
    cost = budget[fin]

    agent_type = 'ortb'
    
    # labels = ['agent_type', 'base_bid', 'bid', 'clicks', 'imps_won', 'total_spend']
    result = [agent_type, bid, bid, n_clk, n_im, cost]

    return result

In [122]:
# Create ORTB Agents: 
ortb_agent_arr = []
# for i in range(agent_num):
temp = ortb_bidding(data)
print(temp)
ortb_agent_arr.append(temp)

['ortb', array([33.09821747, 27.69938627, 30.78142297, ..., 61.79587365,
       39.97369135, 37.87334155]), array([33.09821747, 27.69938627, 30.78142297, ..., 61.79587365,
       39.97369135, 37.87334155]), 96, 87828, 2080269]


In [123]:
# Create final array
final_agent_arr_ortb = []

for item in ortb_agent_arr:    
    bid_values = np.array(item[2])
    final_agent_arr_ortb.append(bid_values)
    
final_agent_arr_ortb = np.array(final_agent_arr_ortb)
print(final_agent_arr_ortb)

[[33.09821747 27.69938627 30.78142297 ... 61.79587365 39.97369135
  37.87334155]]


In [124]:
clk_list = [0] * ((agent_num *5)+1)
bdgt_list  = [limited_budget] * ((agent_num*5)+1)

final_agent_arr = np.concatenate((final_agent_arr_const, final_agent_arr_rand, final_agent_arr_linear, final_agent_arr_sqr, final_agent_arr_exp, final_agent_arr_ortb), axis = 0)
top_index = np.argmax(final_agent_arr,axis=0)

print(final_agent_arr)
print(top_index)
print(len(top_index))

clk = validation['click']
pay = validation['payprice']

[[ 80.          80.          80.         ...  80.          80.
   80.        ]
 [ 85.          85.          85.         ...  85.          85.
   85.        ]
 [ 90.          90.          90.         ...  90.          90.
   90.        ]
 ...
 [111.03410889 102.26095528 106.98894278 ... 219.80159977 125.94204953
  120.88269213]
 [117.56552706 108.27630559 113.28241001 ... 232.73110564 133.35040538
  127.99343872]
 [ 33.09821747  27.69938627  30.78142297 ...  61.79587365  39.97369135
   37.87334155]]
[17 17 17 ... 59 17 17]
303925


In [125]:
# Make clk_list to see who won
for i in range(len(pred)):
    current_agent = top_index[i]
    if pay[i] <= final_agent_arr[top_index[i], i]:
        
        # agent goes above budget
        if pay[i] > bdgt_list[top_index[i]]:
            final_agent_arr[top_index[i], :] = 0
            top_index = np.argmax(final_agent_arr,axis=0)
            
        # add click
        elif final_agent_arr[top_index[i], i] > bdgt_list[top_index[i]]:
            if pay[i] <= bdgt_list[top_index[i]]:
                clk_list[top_index[i]] = clk_list[top_index[i]] + clk[i]
                bdgt_list[top_index[i]] = bdgt_list[top_index[i]] - final_agent_arr[top_index[i], i]

In [126]:
print(clk_list)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]


In [127]:
top_agent_ind = np.argmax(clk_list)

In [128]:
def top_agent_values(index):
    if (index < agent_num):
        return const_agent_arr[index]
    elif (index < agent_num*2):
        return rand_agent_arr[index - agent_num]
    elif (index < agent_num*3):
        return linear_agent_arr[index - (agent_num*2)]
    elif (index < agent_num*4):
        return sqr_agent_arr[index - (agent_num*3)]
    elif (index < agent_num*5):
        return exp_agent_arr[index - (agent_num*4)]
    elif (index < (agent_num*5+1)):
        return ortb_agent_arr[index - (agent_num*5)]
    
print (top_agent_values(top_agent_ind))

['exp', 80, 0           104.502691
1            96.245605
2           100.695476
3            94.405663
4            98.810151
5           115.232149
6           104.722418
7           121.876275
8            96.722480
9           101.714617
10          129.402703
11          110.101278
12           97.868840
13          111.418105
14          110.915659
15           97.857208
16          112.486519
17          115.009057
18          127.842540
19          124.806771
20          123.872503
21          126.603316
22          103.882214
23           98.321503
24          133.815680
25          105.789001
26           89.691379
27          130.122152
28          104.578819
29          214.274564
              ...     
303895    34891.867546
303896      100.317879
303897      133.233878
303898      234.806599
303899      127.853504
303900       92.320164
303901       94.430097
303902      119.157749
303903       90.029384
303904      117.023730
303905      113.459290
303906       98.016054